## Approximate Nearest Neighbours for Stock Market Pattern Search/Recognition
The developed notebook's purpose is to retrieve candlestick patterns in the stock market that has a similar pattern to your input pattern. For this task, approximate nearest neighbor algorithms are used to execute the task of vector similarity search. To feed the search space, you need a CSV file with the following information: 

* Time (YYYY-MM-DD), open, high, low, close. 

The following row is an example of this format 
* 2015-08-07T00:00:00Z, 2.831620, 3.524990, 2.590420, 2.789720

From the fed CSV file, pick an index of any bar and determine a window size. If you want to look at the latest bar's pattern with the previous 20 bars, the index must be 0 and window_size must be 20.

The rest is explained in the following cells.

Note: This is not an API, and this is only meant for practical purposes. Any feedback or comments are welcome.

### Import Libraries
* Pandas: For Dataframes and Data Management
* Annoy: For Vector Similarity Search
* Yahoo Finances: For Data Retrivial
* Plotly: For Visualizations

In [15]:
#!pip install yfinance

In [16]:
import pandas as pd

from annoy import AnnoyIndex

#import investpy
import yfinance as yf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
def get_data(ticker: str):

    msft = yf.Ticker(ticker)
    df = msft.history(period="max")

    # Sort it from newest bar to oldest bar
    df = df.iloc[::-1]

    df = df.drop(["Volume", "Dividends", "Stock Splits"], axis = 1)
    df = df.reset_index()
    df.columns = ["time", "open", "high", "low", "close"]

    # Calculate the daily change
    df["change"] = ( (df.close - df.open) / (df.open * 100) )*10000


    # finalize the change dataframe
    change_df = df[["time", "change"]].reset_index().drop(["index"], axis=1).copy()
    
    return df, change_df

In [18]:
# Add previous bars to the most recent bar in the list for the window of window size
def alter_df(change_df, window_size):
    change_list = change_df["change"].tolist()
    
    # Move elements in the list by 1
    for i in range(window_size-1):
        change_list.pop(0)
        change_list.append(0)
        
        # Assign a column name
        change_df["Candle(" + str(i+1) +")"] = change_list
    
    # Create a list of embeddings
    embeddings = change_df.drop("time", axis=1).values.tolist()
    
    # Get rid of the Null Values
    for i in range(window_size-1):
        embeddings.pop()
        
    return embeddings, change_df

In [36]:
def get_pattern_df(check_for: int, window_size: int, future: bool = False):
    
    """
    check_for: int -> Check for the candle stick at index = check_for from the data frame df
    window_size: int -> Look into past 'window_size' bars. If 20, return 20 previous bars.
    future: bool -> Look into future 'window_size' bars if checked. It will return additional bars if available in the dataset.
    
    Return the dataframe of candlesticks which belongs to the inputted pattern.
    """
    
    # Returned indexes from Annoy
    candles_list = []
    
    # If the user wants to see future bars as well, future must be True
    if future:
        # Return future window_size bars
        for i in range(window_size*2, 0, -1):
            candles_list.append(df.iloc[check_for])
            check_for -= 1
            
        check_for += window_size*2
            
    # Return past window_size bars
    for i in range(window_size):
        candles_list.append(df.iloc[check_for])
        check_for += 1
    
    
    return pd.DataFrame(candles_list)

In [39]:

def visualize_patterns(main: int,
                       window_size: int,
                       closest_n: int,
                       future: bool = False) -> None:
    """
    main: The index of the candle stick where it is the newest bar of the input pattern 
    window_size: Length of the input pattern.
    closest_n: return closest n patterns
    future: Set it to true to see future patterns if available
    
    With plotly, visualize the price graph through candlesticks.
    'Main' is the first subgraph, and the closest ones to the main graph are other subgraphs.
    """
    
    vector_size = len(embeddings[0])

    t = AnnoyIndex(vector_size, 'manhattan')
    for i in range(len(embeddings)):
        t.add_item(i, embeddings[i])

    t.build(100)
    
    nearest_neighs = t.get_nns_by_vector(embeddings[main], closest_n)

    # Remove the duplicates of main bar
    if main in nearest_neighs:
        nearest_neighs.remove(main)
        
    # Get the pattern of the inputted chart
    df_pattern_main = get_pattern_df(main, window_size)
    
    # Create the subgraph spots for down below
    fig = make_subplots(len(nearest_neighs)//2 + 1, cols=2)

    # Print the main graph
    row_i, col_i = 1, 1
    fig.add_trace( go.Candlestick(
                    x=df_pattern_main['time'],
                    open=df_pattern_main['open'],
                    high=df_pattern_main['high'],
                    low=df_pattern_main['low'],
                    close=df_pattern_main['close']),
                    row=1, col=1
                  )
    
    fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)
    
    
    
    col_i += 1
    # Print the closest charts in the form of subgraphs
    for i in nearest_neighs:
        df_pattern_next = get_pattern_df(i, window_size, future)
        fig.add_trace( 
            go.Candlestick(
                x=df_pattern_next['time'],
                open=df_pattern_next['open'],
                high=df_pattern_next['high'],
                low=df_pattern_next['low'],
                close=df_pattern_next['close']
            ),
            row=row_i, col=col_i
        )
        if future:
            fig.add_vline(x = df_pattern_next["time"].iloc[window_size*2], row=row_i, col=col_i)
        else:
            fig.add_vline(x = df_pattern_next["time"].iloc[0], row=row_i, col=col_i)
            
        fig.update_xaxes(rangeslider= {'visible':False}, row=row_i, col=col_i)
        
        col_i = (col_i%2) + 1
        row_i = row_i + 1 if col_i == 1 else row_i
        
            


    fig.update_layout(height=1000, width=1000, xaxis_rangeslider_visible=False)
    fig.show()

In [45]:
# PARAM WINDOW BEGIN ----------------------------

# How many candle sticks you want to look into
window_size = 20
closest_n = 5
input_pattern_index = 0

# PARAM WINDOW END ------------------------------
df, change_df = get_data("BTC-USD")


embeddings, change_df = alter_df(change_df, window_size)

vector_size = len(embeddings[0])

t = AnnoyIndex(vector_size, 'manhattan')
for i in range(len(embeddings)):
    t.add_item(i, embeddings[i])

t.build(100)

visualize_patterns(input_pattern_index,
                   window_size,
                   closest_n,
                   True)
